In [12]:
# Imports
from llmsql.config import Arguments, SettingsManager, Settings
from llmsql.llm import ModelManager
from llmsql.db import Database
from dotenv import load_dotenv

In [13]:
db = Database(db_name="dvdrental",user_name="postgres",pwd="****",host="localhost",port=5432)
sql_database = db.get_sql_db()

Successfully connected to the PostgreSQL database


In [3]:
# dir(sql_database)

In [14]:
tables = sql_database.get_usable_table_names()
tables

['actor',
 'address',
 'category',
 'city',
 'country',
 'customer',
 'film',
 'film_actor',
 'film_category',
 'inventory',
 'language',
 'payment',
 'rental',
 'staff',
 'store']

In [21]:
# load env variables
load_dotenv("../.env")

# Set llm
args = Arguments()

args.llm_type = 'ollama' # gemini ollama deepseek
args.llm_name = 'sqlcoder:latest' # 'deepseek-r1:7b' sqlcoder:latest 'qwen2.5:7b' 'smollm:latest' 'models/gemini-1.5-flash' 'deepseek-chat', 'deepseek-reasoner'
args.temperature = 0.6
args.tokenizer_name = 'defog/sqlcoder-7b-2' # 'deepseek-ai/DeepSeek-R1' 'Qwen/Qwen2.5-7B-Instruct' 'defog/sqlcoder-7b-2'

args.context_window = 8000
args.num_output = 512

args.is_openvino_embed = True
args.embed_model = r"D:\llm-sql\models\kalm_ov"

SettingsManager.set_settings(args=args)

In [22]:
# sanity check
for a in (Settings.llm,Settings.tokenizer,Settings.num_output,Settings.context_window):
    print(a,end="\n\n")

callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x0000022480A2B590> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x000002238B0F4C20> completion_to_prompt=<function default_completion_to_prompt at 0x000002238B1BE980> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None base_url='http://localhost:11434' model='sqlcoder:latest' temperature=0.6 context_window=8000 request_timeout=20000.0 prompt_key='prompt' json_mode=False additional_kwargs={} is_function_calling_model=True keep_alive=None

functools.partial(<bound method PreTrainedTokenizerBase.encode of CodeLlamaTokenizerFast(name_or_path='defog/sqlcoder-7b-2', vocab_size=32016, model_max_length=8000, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'additional_special_tokens': ['▁<PRE>', '▁<MID>', '▁<SUF>', '▁<EOT>', '▁<PRE>', '▁<MID>', '▁<SU

In [23]:
# Build query engine
db.build_query_engine(sql_database=sql_database,
                      tables=sql_database.get_usable_table_names(),
                      sql_table_schemas=None)

query engine has been built.


In [24]:
response = db.query_text("How many unique actors are there?")

In [25]:
str(response)

'1000'

In [26]:
response.metadata['sql_query']

'WITH actor_film AS (SELECT a.actor_id, f.film_id FROM actor a CROSS JOIN film f), film_count AS (SELECT actor_id, COUNT(film_id) AS number_of_films FROM actor_film GROUP BY actor_id) SELECT MAX(number_of_films) FROM film_count;'